In [1]:
import sklearn
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
coffee_clim_data = pd.read_csv(r"..\data\coll_caff_node_clim.csv")
coffee_env_data = pd.read_csv(r"..\data\coll_caff_node_env.csv")

caffeine_content = pd.read_csv(r"..\data\no_caffeine_nodes_w_specimen.csv")

coffee_env_data.head

<bound method NDFrame.head of         specimen_id  source_crs  longitude   latitude  \
0       humbertii_4        4326  43.797500 -22.875000   
1     richardii_256        4326  50.041667 -14.101389   
2      perrieri_261        4326  46.333889 -24.560833   
3      perrieri_262        4326  46.333889 -24.560833   
4      perrieri_263        4326  46.333889 -24.560833   
..              ...         ...        ...        ...   
521   resinosa_2807        4326  47.000000 -23.000000   
522   resinosa_2809        4326  48.700000 -13.580000   
523   dubardii_2812        4326  49.700000 -13.180000   
524   dubardii_2813        4326  49.700000 -13.180000   
525  humbertii_2814        4326  44.580000 -22.920000   

                                  mada_geom_point  sampled_layers  \
0     POINT (376646.4194813568 7469814.360957881)               9   
1    POINT (1044838.0523195444 8435209.185751595)               9   
2     POINT (635084.8917711123 7283025.482374048)               9   
3     POI

In [3]:
caffeine_content

,Species_name,caffeine_percent
0,C_andrambovatensis_A310,0.000
1,C_abbayesii_A601,0.000
2,C_arenesiana_A403,0.000
3,C_bertrandii_A5,0.000
4,C_dubardii_A969,0.000
5,C_heimii_A516,0.000
6,C_humbertii_RNF785,0.000
7,C_millotii_A222,0.000
8,C_perrieri_A12,0.000
9,C_pervilleana_A957,0.000


In [13]:
# Define bin edges and class labels based on your data
bins = [-1, 0.0, 0.5, 100, float('inf')]  # Adjust these values based on your caffeine content distribution
labels = ['Absent', 'Low', 'High', 'Very high']  # Define your class labels

# Assuming 'caffeine_content' is the column with caffeine levels in your dataset
caffeine_content['caffeine_class'] = pd.cut(caffeine_content['caffeine_percent'], bins=bins, labels=labels)

# Check the new column
#print(caffeine_content[['caffeine_percent', 'caffeine_class']])
caffeine_content


,Species_name,caffeine_percent,caffeine_class,extracted_species
0,C_andrambovatensis_A310,0.000,Absent,andrambovatensis
1,C_abbayesii_A601,0.000,Absent,abbayesii
2,C_arenesiana_A403,0.000,Absent,arenesiana
3,C_bertrandii_A5,0.000,Absent,bertrandii
4,C_dubardii_A969,0.000,Absent,dubardii
5,C_heimii_A516,0.000,Absent,heimii
6,C_humbertii_RNF785,0.000,Absent,humbertii
7,C_millotii_A222,0.000,Absent,millotii
8,C_perrieri_A12,0.000,Absent,perrieri
9,C_pervilleana_A957,0.000,Absent,pervilleana


In [14]:
coffee_clim_data['extracted_specimen'] = coffee_env_data['specimen_id'].str.split('_').str[0]
coffee_clim_data.head

<bound method NDFrame.head of         specimen_id  source_crs  longitude   latitude  \
0       humbertii_4        4326  43.797500 -22.875000   
1     richardii_256        4326  50.041667 -14.101389   
2      perrieri_261        4326  46.333889 -24.560833   
3      perrieri_262        4326  46.333889 -24.560833   
4      perrieri_263        4326  46.333889 -24.560833   
..              ...         ...        ...        ...   
521   resinosa_2807        4326  47.000000 -23.000000   
522   resinosa_2809        4326  48.700000 -13.580000   
523   dubardii_2812        4326  49.700000 -13.180000   
524   dubardii_2813        4326  49.700000 -13.180000   
525  humbertii_2814        4326  44.580000 -22.920000   

                                  mada_geom_point  sampled_layers  \
0     POINT (376646.4194813568 7469814.360957881)              70   
1    POINT (1044838.0523195444 8435209.185751595)              70   
2     POINT (635084.8917711123 7283025.482374048)              70   
3     POI

In [15]:
coffee_env_data['extracted_specimen'] = coffee_env_data['specimen_id'].str.split('_').str[0]
coffee_env_data

,specimen_id,source_crs,longitude,latitude,mada_geom_point,sampled_layers,nodata_layers,is_categorical_encoded,env_71_alt_Altitude (meters),env_72_slo_Slope (degrees),env_73_asp_Aspect; clockwise from North (degrees),"env_74_solrad_Solar radiation; computed from altitude, slope and aspect (Wh.m-2.day-1)","env_75_geo_Rock types (categ_vals: 1, 2, 4, 5, 6, 7, 9, 10, 11, 12, 13)","env_76_soi_Soil types (categ_vals: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23)","env_77_veg_Vegetation types (categ_vals: 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 22, 23, 25)","env_78_wat_Watersheds (categ_vals: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25)",env_79_forcov_Percentage of forest cover in 1 km by 1 km grid cells (%),extracted_specimen
0,humbertii_4,4326,43.797500,-22.875000,POINT (376646.4194813568 7469814.360957881),9,0,False,477.0,11.0,346.0,7929.0,5.0,15.0,12,6.0,68.0,humbertii
1,richardii_256,4326,50.041667,-14.101389,POINT (1044838.0523195444 8435209.185751595),9,0,False,261.0,13.0,7.0,8107.0,10.0,6.0,14,1.0,87.0,richardii
2,perrieri_261,4326,46.333889,-24.560833,POINT (635084.8917711123 7283025.482374048),9,0,False,253.0,1.0,247.0,7366.0,10.0,18.0,7,6.0,0.0,perrieri
3,perrieri_262,4326,46.333889,-24.560833,POINT (635084.8917711123 7283025.482374048),9,0,False,253.0,1.0,247.0,7366.0,10.0,18.0,7,6.0,0.0,perrieri
4,perrieri_263,4326,46.333889,-24.560833,POINT (635084.8917711123 7283025.482374048),9,0,False,253.0,1.0,247.0,7366.0,10.0,18.0,7,6.0,0.0,perrieri
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,resinosa_2807,4326,47.000000,-23.000000,POINT (704992.1724866081 7455081.895663038),9,0,False,544.0,5.0,201.0,7607.0,10.0,10.0,7,17.0,0.0,resinosa
522,resinosa_2809,4326,48.700000,-13.580000,POINT (900530.2787016315 8495683.113346167),9,0,False,40.0,7.0,292.0,7875.0,6.0,18.0,5,11.0,39.0,resinosa
523,dubardii_2812,4326,49.700000,-13.180000,POINT (1009820.5473967544 8538182.939559158),9,0,False,171.0,8.0,36.0,7888.0,10.0,9.0,6,1.0,24.0,dubardii
524,dubardii_2813,4326,49.700000,-13.180000,POINT (1009820.5473967544 8538182.939559158),9,0,False,171.0,8.0,36.0,7888.0,10.0,9.0,6,1.0,24.0,dubardii


In [16]:
caffeine_content['extracted_species'] = caffeine_content['Species_name'].str.split('_').str[1]
caffeine_content

,Species_name,caffeine_percent,caffeine_class,extracted_species
0,C_andrambovatensis_A310,0.000,Absent,andrambovatensis
1,C_abbayesii_A601,0.000,Absent,abbayesii
2,C_arenesiana_A403,0.000,Absent,arenesiana
3,C_bertrandii_A5,0.000,Absent,bertrandii
4,C_dubardii_A969,0.000,Absent,dubardii
5,C_heimii_A516,0.000,Absent,heimii
6,C_humbertii_RNF785,0.000,Absent,humbertii
7,C_millotii_A222,0.000,Absent,millotii
8,C_perrieri_A12,0.000,Absent,perrieri
9,C_pervilleana_A957,0.000,Absent,pervilleana


In [19]:
merged_df = pd.merge(
    coffee_env_data,
    caffeine_content[['extracted_species', 'caffeine_class']],
    left_on='extracted_specimen',
    right_on='extracted_species',
    how='left'  # Use 'left' join to keep all rows from coffee_env_data
)

# Step 4: Drop the helper columns
merged_df = merged_df.drop(columns=['extracted_specimen', 'extracted_species'])

# The result is the original DataFrame with an additional 'caffeine_class' column
merged_df.to_csv(r'../data/coll_caff_node_env_w_class.csv', index=False)

In [20]:
merged_df = pd.merge(
    coffee_clim_data,
    caffeine_content[['extracted_species', 'caffeine_class']],
    left_on='extracted_specimen',
    right_on='extracted_species',
    how='left'  # Use 'left' join to keep all rows from coffee_env_data
)

# Step 4: Drop the helper columns
merged_df = merged_df.drop(columns=['extracted_specimen', 'extracted_species'])

# The result is the original DataFrame with an additional 'caffeine_class' column
merged_df.to_csv(r'../data/coll_caff_node_clim_w_class.csv', index=False)